This code refers to EXALT baseline: https://github.com/pranaydeeps/WASSA24_EXALT

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import time
from transformers_interpret import SequenceClassificationExplainer
import torch
from typing import Dict, List, Optional, Tuple, Union
from transformers import PreTrainedModel, PreTrainedTokenizer

class CustomSequenceClassificationExplainer(SequenceClassificationExplainer): # need custom explainer to handle xlm-roberta
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        attribution_type: str = "lig",
        custom_labels: Optional[List[str]] = None,
    ):
        super().__init__(model, tokenizer)
        
    def _make_input_reference_token_type_pair(self, input_ids: torch.Tensor, sep_idx: int = 0
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Returns two tensors indicating the corresponding token types for the `input_ids`
        and a corresponding all zero reference token type tensor.
        Args:
            input_ids (torch.Tensor): Tensor of text converted to `input_ids`
            sep_idx (int, optional):  Defaults to 0.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]
        """
        seq_len = input_ids.size(1)
        
        if self.model.config.model_type == 'xlm-roberta':
            token_type_ids = torch.zeros(seq_len, dtype=torch.int, device=self.device).expand_as(input_ids)
        else:
            token_type_ids = torch.tensor([0 if i <= sep_idx else 1 for i in range(seq_len)], device=self.device).expand_as(
                input_ids
            )
        ref_token_type_ids = torch.zeros_like(token_type_ids, device=self.device).expand_as(input_ids)

        return (token_type_ids, ref_token_type_ids)


def Clean_AttributionTokens(tokenized_text, attributions):
    """creates a vector of binary values to indicate whether a word is a trigger word or not (based on a predefined threshold)

    Args:
        normalized_scores (list): a list of numerical scores that have already been normalized (i.e., they sum to 1.0)
        threshold (float, optional): A lower bound for converting numerical values to a binary 1. Values below the threshold are converted to 0. Defaults to 0.2.

    Returns:
        list: a binary vector of 1s and 0s indicating whether a word is a trigger word or not
    """
    offset_mapping = tokenizer(tokenized_text, return_offsets_mapping=True)["offset_mapping"]
    #print(offset_mapping)
    final_attributions = {}

    # to ensure the same mapping, we need to find the indices of the spaces (which are token+1)
    space_indices = [] # counting first character as a space because otherwise the first token will be skipped

    # keep track of the space indices
    for char_index, character in enumerate(tokenized_text + " "):  # add a space to capture the final token
        #print(char_index, character)
        if character.isspace():
            #print("Space found", char_index)
            space_indices.append(char_index)

    # not very effective to run over ALL mappings for EACH token, but it works
    # for each space (i.e. token), find the corresponding sub-tokens and sum the attributions based on character
    for i, space_index in enumerate(space_indices, start =0):
        final_attributions[i] = 0
        for tokenindex, mapping in enumerate(offset_mapping):
            begin_index = mapping[0]
            end_index = mapping[1]
            if begin_index == 0 and end_index == 0: # ignore BoS and EoS tokens
                continue
            elif i == 0: # special treatment because there is no previous token for the first token
                if space_index >= end_index: # any sub-tokens before space index (token delimiter) are concatenated 
                    final_attributions[i] += attributions[tokenindex]
            
            else:
                if space_index >= end_index and begin_index >= space_indices[i-1]: # begin index > previous space index because otherwise importances will overlap
                    final_attributions[i] += attributions[tokenindex]
                elif space_index < begin_index:
                    break
    
    #print(final_attributions)
    final_outputs = []
    for key in final_attributions.keys():
        final_outputs.append(final_attributions[key])
    return tokenized_text.split(" "), final_outputs

def Vector_from_raw_attributions(inputstring, interpret_output, threshold=0.1):
    """ a combined function to normalize, clean, and create a binary vector from raw numerical attributions values for subtokens
    Args:
        inputstring (string): the input text string (from the column ["Texts"])
        interpret_output (Tuple): a Tuple containing the output from the importance attribution model (i.e., the raw attributions + the subtokens)
        threshold (float, optional): Minimal required importance to convert the numerical value to a binary 1. Defaults to 0.1.

    Returns:
        list: a vector of binary values indicating whether a token is a trigger word or not
    """
    sub_tokens, attribute_scores = zip(*interpret_output)
    tokenized_sample, attribute_scores = Clean_AttributionTokens(inputstring, attribute_scores)
    return tokenized_sample



In [3]:
# to run this on the entire dataset
import pandas as pd
import os

# pick the model you want to use
MODEL_NAME = ''

# load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to('cuda:0')
cls_explainer = CustomSequenceClassificationExplainer(model, tokenizer)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at /work/share/zjh/Huize/test/infer-hw/__pycache__/emo/resultT/xlm-roberta-large=exalt_emotion_train=fullPT=lr4e5=bs256=rand=bf16=0510-231202/checkpoint-387 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
traindata = pd.read_csv("data/raw/exalt_triggers_train.tsv", sep="\t")

tokens = []
for rowindex, row in traindata[:].iterrows():
    tweet_text = row["Texts"]
    interpret_output = cls_explainer(tweet_text)
    final_tokens = Vector_from_raw_attributions(tweet_text, interpret_output,thr)
    tokens.append(final_tokens)
    break

print(final_tokens)
# traindata["tokens"] = tokens
# traindata.to_csv("data/raw/exalt_triggers_test_participants_token.tsv", sep="\t", index=False)

['@user', 'I’m', 'so', 'happy', 'you’ve', 'found', 'some', 'success', 'for', 'yourself']
